In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import torch
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import time
import os
import json
from scipy.special import softmax

[nltk_data] Downloading package punkt to
[nltk_data]     /home/p.bushipaka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
train = pd.read_csv('../data/train.csv', index_col=0)
val = pd.read_csv('../data/val.csv', index_col=0)
test = pd.read_csv('../data/test.csv', index_col=0)

In [7]:
print(train.shape)
print(val.shape)
print(test.shape)

(734, 17)
(157, 17)
(157, 17)


In [8]:
train['label'] = train[['answer1', 'answer2', 'answer3']].apply(lambda x: x.dropna().tolist(), axis=1)
train = train.explode('label').reset_index(drop=True)

val['label'] = val[['answer1', 'answer2', 'answer3']].apply(lambda x: x.dropna().tolist(), axis=1)
val = val.explode('label').reset_index(drop=True)

In [9]:
print(train.shape)
print(val.shape)

(2199, 18)
(471, 18)


In [10]:
labels = train['label'].unique().tolist()
labels = [s.strip() for s in labels ]


NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [11]:
print(id2label)
print(label2id)

{0: 'Pro', 1: 'Neutral', 2: 'Not-about', 3: 'Against', 4: 'Link-broken'}
{'Pro': 0, 'Neutral': 1, 'Not-about': 2, 'Against': 3, 'Link-broken': 4}


In [12]:
train["labels"] = train['label'].map(lambda x: label2id[x.strip()])
val["labels"] = val['label'].map(lambda x: label2id[x.strip()])

In [13]:
train_df = train[['Input', 'labels']] 
eval_df = val[['Input', 'labels']]


In [14]:
save_dir = '../output/'
model_name = 'FacebookAI/roberta-base'
model_name_filename = model_name.replace("/", "-")

In [15]:
start_time = time.time()
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.INFO)

from simpletransformers.classification import ClassificationModel, ClassificationArgs

# giving max_input_length (based on model) and batch size
max_seq_length = 512
batch_size = 32 # change it if there is Out of Memory (OOM) error



# setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 4



model_args.overwrite_output_dir = True
model_args.best_model_dir = f'./output/{model_name_filename}_2ndmodel/best_model'
model_args.reprocess_input_data = True


model_args.evaluate_during_training = True
model_args.early_stopping_metric = 'eval_loss'  # eval_loss The metric that should be used with early stopping. (Should be computed during eval_during_training)
model_args.use_early_stopping = True
model_args.max_seq_length = max_seq_length
model_args.manual_seed = 42 # for reproducible results
model_args.do_lower_case = True # when we are using BERT Base/large uncased. converts to lower case
model_args.train_batch_size = batch_size
model_args.eval_batch_size = batch_size

model_args.save_eval_checkpoints =  False
model_args.save_model_every_epoch = False
#model_args.sliding_window = True #for chunking
#model_args.stride = 100  # this is how much overlap we need between two chunks
#model_args.tie = 1
model_args.save_to_hub = True

# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "FacebookAI/roberta-base",
    num_labels=5,
    args=model_args,
    use_cuda=True,
    cuda_device=0,
    
)

# Train model
_, training_details = model.train_model(train_df, eval_df = eval_df)


# show training details
logging.info('training details')
logging.info(pd.DataFrame(training_details))

model = ClassificationModel(model.args.model_type, model.args.best_model_dir)

end_time = time.time()
total_time = end_time - start_time
total_time_minutes, total_time_seconds = divmod(total_time, 60)
print(f'total time taken for the finetuning of {model_name}: {total_time}')
time_data = {
        "total_time": {
            "minutes": int(total_time_minutes),
            "seconds": round(total_time_seconds, 2)
        }
}
t_filename = f"time_{model_name_filename}.json"
t_filepath = os.path.join(save_dir, t_filename)
with open(t_filepath, 'w') as json_file:
    json.dump(time_data, json_file)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /FacebookAI/roberta-base/resolve/main/config.json HTTP/1.1" 200 0
loading configuration file config.json from cache at /home/p.bushipaka/.cache/huggingface/hub/models--FacebookAI--roberta-base/snapshots/e2da8e2f811d1448a5b465c236feacd80ffbac7b/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hid

  0%|          | 0/2199 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_512_5_2


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/69 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/69 [00:00<?, ?it/s]

/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/471 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_5_2
Configuration saved in outputs/config.json
Model weights saved in outputs/model.safetensors
tokenizer config file saved in outputs/tokenizer_config.json
Special tokens file saved in outputs/special_tokens_map.json
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:root:training details
INFO:root:   global_step  train_loss       mcc  eval_loss
0           69    1.717391  0.140834   1.340700
1          138    1.114826  0.178009   1.261465
2          207    0.991763  0.233648   1.249095
3          276    0.930340  0.286821   1.250921
loading configuration file ./output/FacebookAI-roberta-base_2ndmodel/best_model/config.json
Model config RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_pro

total time taken for the finetuning of FacebookAI/roberta-base: 218.95101284980774


In [16]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)

/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/471 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_5_2


Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.23364788239041318, 'f1': 0.445859872611465, 'acc': 0.445859872611465, 'eval_loss': 1.2490947326024373}


### prediction on test set

In [17]:
test = pd.read_csv('../data/test.csv', index_col=0)
print(test['docID'].nunique())
print(test.shape)
#test['label'] = test[['answer1', 'answer2', 'answer3']].apply(lambda x: x.dropna().tolist(), axis=1)
#test = test.explode('label').reset_index(drop=True)

157
(157, 17)


In [18]:
labels = train['label'].unique().tolist()
labels = [s.strip() for s in labels ]


NUM_LABELS= len(labels)

id2label={id:label for id,label in enumerate(labels)}

label2id={label:id for id,label in enumerate(labels)}

In [19]:
save_dir = '../output/'
model_name = 'FacebookAI/roberta-base'
model_name_filename = model_name.replace("/", "-")

In [11]:
model = ClassificationModel(
    "bert", f'./output/{model_name_filename}_2ndmodel/best_model', use_cuda = True
)

In [20]:
def make_predictions(row):
    doc = row['Input']
    predictions, raw_outputs = model.predict(doc)
    prediction_id = predictions[0]
    probabilities = softmax(raw_outputs, axis=1)
    # Map prediction ID to label
    predicted_label = id2label[prediction_id]
    # Create dictionary of labels and probabilities
    prediction_dict = {label: prob for label, prob in zip(id2label.values(), probabilities[0])}
    # Return both prediction and probabilities
    return predicted_label, probabilities[0], prediction_dict

predictions = test.apply(make_predictions, axis=1, result_type='expand')
test = test.assign(
    predicted_label=predictions[0],
    probabilities=predictions[1],
    prediction_dict=predictions[2]
)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5908 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/13172 [00:00<?, ?it/s]

...

  0%|          | 0/8893 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2696 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
mode_predictions = test.groupby('docID')['predicted_label'].apply(lambda x: x.mode()[0]).reset_index()

# Calculate sum of predicted probabilities for each class for each docID
sum_probabilities = test.groupby('docID')['probabilities'].apply(lambda x: np.sum(x)).reset_index()

# Merge mode predictions and sum probabilities back to the test DataFrame
test = pd.merge(test, mode_predictions, on='docID', suffixes=('', '_mode'))
test = pd.merge(test, sum_probabilities, on='docID', suffixes=('', '_sum'))

In [22]:
test.to_csv('./results/mp_roberta_results.csv', index = False)

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

y_true = test['majority_label']
y_pred = test['predicted_label']

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy*100)

# Precision
precision = precision_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Precision:", precision*100)

# Recall
recall = recall_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("Recall:", recall*100)

# F1 Score
f1 = f1_score(y_true, y_pred, average='macro')  # 'macro' averaging for multiclass
print("F1 Score:", f1*100)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_true, y_pred)
print("Classification Report:")
print(class_report)


Accuracy: 47.77070063694268
Precision: 44.278290147855365
Recall: 43.6313819834269
F1 Score: 41.4387031408308
Confusion Matrix:
[[16 13  0  0  0]
 [ 5 30  0  5  3]
 [ 5  9  0  3  1]
 [ 3  6  0 14  1]
 [ 4 23  0  1 15]]
Classification Report:
              precision    recall  f1-score   support

     Against       0.48      0.55      0.52        29
     Neutral       0.37      0.70      0.48        43
 No majority       0.00      0.00      0.00        18
   Not-about       0.61      0.58      0.60        24
         Pro       0.75      0.35      0.48        43

    accuracy                           0.48       157
   macro avg       0.44      0.44      0.41       157
weighted avg       0.49      0.48      0.45       157



/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/p.bushipaka/miniconda3/envs/hlt/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.